# Lab 1 - The Basics of Modelling with IBM ILOG CP Optimizer


Throughout the different tutorials, you should consult regularly the documentation [`docplex` constraint programming documentation](http://ibmdecisionoptimization.github.io/docplex-doc/cp/index.html).


You need to run the following code for configuration each time you start the kernel

In [1]:
from docplex.cp.model import *
context.solver.agent = 'local'
#For a personnel machine: 
#context.solver.local.execfile = '/Users/msiala/Applications/CPLEX_Studio2211/cpoptimizer/bin/arm64_osx/cpoptimizer'
#If you use the solver in INSA
context.solver.local.execfile = '/usr/local/insa/ibm_cplex_studio_2211/cpoptimizer/bin/x86-64_linux/cpoptimizer'
context.params.set_attribute('Presolve', 'Off')
context.params.set_attribute('Workers', 1)


## Hello World

Let's start with a simple problem. Consider the following Constraint Satisfaction Problem: 

- Variables: $x$, $y$, $z$
- Domains: $\cal{D}(x) = \cal{D}(y) = \cal{D}(z) = \left\{1, 2, 3\right\}$ for each variable
- Constraints: $x \ne y$, $x \ne z$, $y \ne z$


- We first need to import [`CpoModel`](http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.model.py.html#docplex.cp.model.CpoModel) from `docplex.cp.model` and create an instance:

```python
from docplex.cp.model import CpoModel

mdl = CpoModel(name='My first docplex model')
```

- Next, we create the different variables using [`CpoModel.integer_var`](http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.expression.py.html#docplex.cp.expression.integer_var), [`CpoModel.integer_var_list`](http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.expression.py.html#docplex.cp.expression.integer_var_list) or [`CpoModel.integer_var_dict`](http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.expression.py.html#docplex.cp.expression.integer_var_dict).

For instance: 
```python
x, y, z = mdl.integer_var_list(3, 1, 3, 'x')
```

- Finally, we add the constraints using [`CpoModel.add`](http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.model.py.html#docplex.cp.model.CpoModel.add). 

```python
mdl.add(x != y)
```

In [2]:

# Create the model
mdl = CpoModel()

# Create a list of 3 variables with domain [1 .. 3]
x, y, z = mdl.integer_var_list(3, 1, 3, 'L')

# Add some constraints
mdl.add(x != y)
mdl.add(y != z)
mdl.add(z != x)


- Use [`CpoModel.solve`](http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.model.py.html#docplex.cp.model.CpoModel.solve) to solve the model:

```python
sol = mdl.solve()
```

- Use [`CpoSolveResult.print_solution`](http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.solution.py.html#docplex.cp.solution.CpoSolveResult.print_solution) to print a solution

```python
sol.print_solution()
```


In [ ]:
#code here
# Solve the model
sol = mdl.solve()

# Print the solution
sol.print_solution()


- Value retrieval: [`CpoSolveResult.get_value`](http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.solution.py.html#docplex.cp.solution.CpoSolveResult.get_value) or `CpoSolveResult.__getitem__` 

```python
value_of_x = sol.get_value('x0')
```
Or
```python
value_of_x = sol[x]
```
                                   

In [ ]:
#Find the values of x, y, and z
value_of_x = sol.get_value('L_0')
print ("value of x is " ,  value_of_x)

value_of_x = sol[x]
print ("value of x is " ,  value_of_x)

#...

We can use sol.get_solver_log() to get the solver log and sol.get_solver_infos() to get all the statistics. Note that sol.get_solver_infos()  returns a __dictionary__. 

We can also check the search status via sol.get_solve_status() 

In [ ]:

sol = mdl.solve()

print("Status "  ,  sol.get_solve_status() ) 
print(sol.get_solver_log())
print(sol.get_solver_infos() )


You can call sol.get_solver_infos() to get the different statistics such as the run time, the number of nodes, the number of failures, etc.

In [ ]:
print("nb decisions "  , sol.get_solver_infos()['NumberOfChoicePoints'] )
print("nb fails "  ,  sol.get_solver_infos()['NumberOfFails'] )
print("Total runtime "  ,  sol.get_solver_infos()['TotalTime'] )

We can also ask to generate all the solutions as follows: 

In [ ]:
for sol in mdl.start_search():
    print('x={}, y={}, z={}'.format(sol[x], sol[y], sol[z]))


## Exercice 1: The AllDifferent Global Constraint 



Consider the following CSP: 

- Variables $x_1, x_2, \ldots x_n$
- Domains: $\forall i \in [1,n], \cal{D}(x_i) = \{1, 2 , \ldots n-1\}$ 
- Constraints: $\forall i \neq j,  x_i \ne x_j$


Without using the solver, is this problem satisfiable? Why? 

Write a function model_decomposition(n) that takes as input an integer $n$ and returns a model that uses only binary inequalities (i.e., no global constraints)

Call this function for $n= 9$ then solve this problem. How many nodes did the solver explore? What is the CPU time? 

Write a function $model\_using\_alldiff(n)$ that takes as input an integer $n$ and returns the CSP model of this problem using only one ALLDifferent constraint (consult the all_diff constraint in the documentation).

Call this function for  n=9  then solve this problem. How many nodes did the solver explore? Is there a theoretical reason ? 

Do you start to appretiate global constraints? Why?  

We will evaluate properly the two models on a larger scale. For this reason we increase the value of $n$ from 3 to $30$ and plot the runtime and the number of nodes for each model. 

In order to keep the runtime under control, we will limit the solver to 15 seconds per call. Have a look at the different parameters that can be used in the solve() function. A better and modular way to tune the solving is to use the class CpoParameters. 

Example : 

```
from docplex.cp.parameters import CpoParameters

params = CpoParameters(TimeLimit=10, LogVerbosity=‘Quiet’, LogPeriod=900000)

••• 
sol = model.solve(TimeLimit= params.TimeLimit , LogVerbosity = params.LogVerbosity )


```


When a solver hits the time timit, it will simply stop the search and says that it couldn't solve the problem within the time limit. 

To switch off the verbosity completely you should add 

```
context.verbose = 0
```
along with LogVerbosity='Quiet'

Write a function $run(model, params)$ that run the solver on a given model $m$ using parameters $p$. The function returns the tuple of statistics [number of nodes, total runtime, search status]

Using the $run(m, p)$ function, run the two models model_decomposition(n) and model_using_alldiff(n) for $n \in [3,30]$. Compare the performances of these models (based on the runtime and the search size)

What's your overall impression ? what did you learn so far ? 

## Exercice 2: The Queens & The Art of Branching 

You are given an N-by-N chessboard, and your goal is to place N chess queens on it so that no two queens threaten each other. That is, a solution requires that no two queens share the same row, column or diagonal.

<div class="row" style="margin-top: 10px">
    <div class="col-md-5">
        <img src="display/images/empty-chessboard.png" style="margin-right: 0; width: 160px;" />
    </div>
    <div class="col-md-2" style="display: table">
        <i class="fa fa-arrow-right" style="display: table-cell; font-size: 50px; 
        margin: auto; text-align: center; vertical-align: middle; height: 150px"></i>
    </div>
    <div class="col-md-5">
        <img src="display/images/nqueens8-chessboard.png" style="margin-left: 0; width: 160px;" />
    </div>
</div>



**Exercice**: Create a function `decomposition_model(N)` that models the problem using only binary inequality constrants (no global constraint) and returns an instance of `CpoModel` for the n-queens problem with `N` queens.

Test your function by solving the n-queens problem for small values of $N$ ($< 20$) then print the solution.

**Question**: How many solutions are there for $N = 3,~\ldots,~10$? 

**Note:** To answer this question, you must force the solver to use a depth first strategy: ... start_search(SearchType='DepthFirst')


### Second model with global constraints

Create a function `global_constraint_model(N)` that models and returns an instance of `CpoModel` for the n-queens problem with `N` queens, using **only** and exaclty 3 global constraints.

Test your function by solving the n-queens problem for small values of  N  ( <20 ).

How many solutions are there for  $N=3, .., 10$ ? (It must be the same number as the previous model)

### Branching strategies


Recall that a CP solver is a backtracking solver. At each node, it applies filtering (also called propagation or pruning), then makes a decision about the next node to explore. This decision is a pure heuristic choice. That is, it could be a wrong decision. It's only based on intuition. 

In CP, a decision usually follows the following steps: 
- Choose an unassigned variable $x$
- Choose a value $v$ from the current domain of $x$, and assign $v$ to $x$. 

The above steps require a variable heuristic and a value heuristic. This is what we call a __branching strategy__. 

Branching strategies can be generic (strategies that can be used for any problem), or specific (designed for a specific problem). In CPOptimizer, there are a number of genereic strategies offered. This concerns both variable and value heuristics. 


For example, if $L$ is the list of decision variables, we can use a variable heuristic that picks the variable $x$ with the smallest domain size, and assigns a random value from its domain to it. For this, we need first to declare and add a search_phase as follows: 
```
SearchPhase= model.search_phase(L, 
                                varchooser=model.select_smallest(model.domain_size()),
                                valuechooser=model.select_random_value())

model.add_search_phase(SearchPhase)
```

Read about the different search strategies and how to use them here: 
http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#search-phases




We want to evaluate some strategies from the following list: 

For variable evaluators, you can use:  
 - domain_size() 
 - var_impact()
 - var_local_impact() 
 - var_index()
 
For value evaluators, you can use: 
 - value_impact()
 - value_index()
 
As for the selectors you can use: 
- select_smallest() 
- select_largest ()


You can also use random heuristics. 

__Question__: Using the global constraints model, evaluate __two__ different strategies of your choice for different values of $N$ (in terms of the search space and the runtime).

#### Proposed setting: 
- Time limit 10s for each call 
- Start from n=500 with a step size 20
- No log for faster execution  (for instance by using LogVerbosity='Quiet')
- SearchType='DepthFirst' (We will investigate deeply this choice in the next tutorial)

Is this what you expect? Is the choice of the branching strategy important? Justify

What did you learn today ? 